In [3]:
import eigen_rootfinding as er
import numpy as np
from scipy import linalg as la
from matplotlib import pyplot as plt
import time
from tests.variations import *
%load_ext autoreload
%autoreload 2

In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
methods =     ['qrpmac',
               'svdmac',
               'lqmac',
               'qrpnull',
               'svdnull',
               'lqnull',
               'qrpmac_randcombs',
               'svdmac_randcombs',
               'lqmac_randcombs',
               'qrpnull_randcombs',
               'svdnull_randcombs',
               'lqnull_randcombs',
               'qrpfastnull',
               'svdfastnull',
               'lqfastnull']

In [6]:
np.random.seed(0)
deg = 4
dim = 3
polys = [er.polynomial.getPoly(deg,dim,power=False) for polynum in range(dim)]

In [7]:
for method in methods:
    randcombos = method[-5:] == 'combs'
    if randcombos: method = method[:-10]
    roots = er.solve(polys,method=method,randcombos=randcombos)
    rootsrt = np.argsort(roots[:,0])
    if randcombos: method += '_randcombs'
    print("{:<20}".format(method),
          report_residuals(polys,
                           roots[rootsrt],
                           kinds=['max']),
          sep='\t')

qrpmac              	{'max': 2.0254372917896027e-13}
svdmac              	{'max': 1.8285968423053002e-14}
lqmac               	{'max': 1.8811118860719502e-14}
qrpnull             	{'max': 6.27970172589564e-14}
svdnull             	{'max': 1.0934582254498909e-14}
lqnull              	{'max': 1.470147567306465e-14}
qrpmac_randcombs    	{'max': 2.872644901621895e-13}
svdmac_randcombs    	{'max': 1.2177246315510973e-13}
lqmac_randcombs     	{'max': 9.090237599824796e-14}
qrpnull_randcombs   	{'max': 1.6650466263183473e-12}
svdnull_randcombs   	{'max': 4.794556571245187e-13}
lqnull_randcombs    	{'max': 4.804787541301085e-13}
qrpfastnull         	{'max': 4.048102881804567e-14}
svdfastnull         	{'max': 1.2938073009559383e-14}
lqfastnull          	{'max': 8.36485431047552e-15}


In [ ]:
#todo turn this into a function... in a way that's easy to run on the server
dims = [2,3]
degs = np.arange(2,8)
numdegs = len(degs)
nummethods = len(methods)
avgresiduals = np.zeros((dim,nummethods,numdegs))
negmeanlog10s = np.zeros((dim,nummethods,numdegs))
maxresiduals = np.zeros((dim,nummethods,numdegs))
timing = np.zeros((dim,nummethods,numdegs))
for dim in dims:
    for deg in degs:
        data = run_tests(deg,dim,methods=methods)
        for methodnum,method in enumerate(methods):
            avgresiduals[dim-min(dims),methodnum,deg-min(degs)] = np.mean(data[method]['raw'])
            negmeanlog10s[dim-min(dims),methodnum,deg-min(degs)] = np.mean(data[method]['log10'])
            maxresiduals[dim-min(dims),methodnum,deg-min(degs)] = np.max(data[method]['max'])
            timing[dim-min(dims),methodnum,deg-min(degs)] = np.mean(data[method]['time'])

In [ ]:
alpha = .75
dim = 2
skipmethods = {'qrpmac',
               'qrpmac_randcombs',
               'qrpnull_randcombs',
               'qrpnull',
               'qrpfastnull',
               'lqmac_randcombs',
               'svdmac_randcombs',
               'lqnull_randcombs',
               'svdnull_randcombs',
#                'svdmac',
#                'lqmac'
                }
colors = [f'C{i}' for i in range(10)]+['k','coral','grey','navy','chartreuse']
assert len(colors) >= len(methods),(len(colors),len(methods))
data = [avgresiduals[dim-2],10**negmeanlog10s[dim-2],maxresiduals[dim-2],timing[dim-2]]
datanames = ['avgresiduals','average log residuals','maxresiduals','timing']
for data,dataname in zip(data,datanames):
    plt.figure(figsize=(7,7))
    for methodnum,(method,color) in enumerate(zip(methods,colors)):
        if method in skipmethods:
            continue
        plt.semilogy(degs,
                     data[methodnum],
                     color,
                     label=method,
                     alpha=alpha)
    plt.title(dataname)
    plt.legend()
    plt.show()